### DotDict class

What I want it to do:
- works with any dictionary whose keys are strings (and whose nested dictionaries keys are strings as well)
- `dotdict_test = DotDict(dict_test)` gives a DotDict object, `dict_test = dict(dotdict_test)` gives back the dictionary
- can access the entries through the "dot" notation AND the regular dictionary index
- setting / accessing / deleting an attribute sets / accesses / deletes the same entry in the dictionary
- can create an empty dotdict with `DotDict()`

In [448]:
%reset -f
import re
from typing import Optional, Any


class KeyTypeError(Exception):
    """
    A custom exception for the DotDict class. Raised when a trying to use
    DotDict with a key that is not a string.
    """


class AttributeNameError(Exception):
    """
    A custom exception for the DotDict Class. Raised when trying to use
    DotDict with a key that cannot with used as an attribute name.
    """


# best attempt at defining a Regular Expression for an attribute name
attribute_name_pattern_string = "^[a-zA-Z]\w*$"
attribute_name_pattern = re.compile(attribute_name_pattern_string)


def check_key(key):
    # check if the key is a string
    if not (type(key) is str):
        raise KeyTypeError(f"The key '{key}' is not a string")
    # check if the key is accessible as a not-hidden attribute
    if not attribute_name_pattern.match(key):
        raise AttributeNameError(f"The key '{key}' cannot be used as an attribute name")


def check_keys(dict_to_check: dict):
    """
    Recursively checks if the keys of dict_to_check and its
    nested dictionaries are accessible as attribute names.
    """
    for key in dict_to_check.keys():
        check_key(key)
        # if the value is a dict, check for its keys as well
        value = dict_to_check[key]
        if type(value) is dict:
            check_keys(value)


def clean_types(dict_to_check: dict) -> dict:
    """
    Recursively converts the types of all the values of dict_to_check and its
    nested dictionaries to DotDicts when they are dicts.
    """
    # placeholder for the clean dict
    cleaned_dict = {}
    for key in dict_to_check.keys():
        value = dict_to_check[key]
        # convert the value to a dict if it is a DotDict
        if type(value) is DotDict:
            value = dict(value)
        # clean the types of the value if it is a dict
        if type(value) is dict:
            value = clean_types(value)
        # add the clean value to the clean dict
        cleaned_dict[key] = value
    # return the clean dict
    return cleaned_dict


class DotDict(dict):
    """
    Allows handling dictionaries with the "dot" notation.
    """

    def __init__(self, *args, _check: bool = True, **kwargs):
        if _check:
            # create a temporary dict from the arguments
            dict_self = dict(*args, **kwargs)
            # test if all keys are valid as attribute names, recursively
            check_keys(dict_self)
            # if any values are DotDicts, convert them to dict, recursively
            dict_self = clean_types(dict_self)
            # init the "self" dict with the cleaned dict
            dict.__init__(self, dict_self)
        else:
            # init the "self" dict from the arguments directly
            dict.__init__(self, *args, **kwargs)

    def __getitem__(self, key: str) -> Optional[Any]:
        # check if the key exists
        if not (key in self.keys()):
            raise AttributeError(f"No key/attribute '{key}'")
        value = dict.__getitem__(self, key)
        if type(value) is dict:
            # if the value is a dict, return it as a DotDict object
            # no need to check it, it's already cleared
            return DotDict(value, _check=False)
        else:
            return value

    __getattr__ = __getitem__

    def __setitem__(self, key: str, value: Optional[Any]):
        # check if the key is valid as an attribute name
        check_key(key)
        # if the value is a dict, convert it to DotDict (clears/cleans it)
        if type(value) is dict:
            value = DotDict(value)
        # if the value is a DotDict, insert it as a dict (already cleared)
        if type(value) is DotDict:
            value = dict(value)
        dict.__setitem__(self, key, value)

    __setattr__ = __setitem__

    def __delitem__(self, key: str):
        # check if the key exists
        if not (key in self.keys()):
            raise AttributeError(f"No key/attribute '{key}'")
        dict.__delitem__(self, key)

    __delattr__ = __delitem__

    def __dir__(self) -> list[str]:
        # overridden to show the keys as attributes with the 'dir' built-in function
        dict_dir = dict.__dir__(self)
        dotdict_attributes = list(self.keys())
        return dict_dir + dotdict_attributes

In [449]:
test = DotDict()
test.lol = 3
test

{'lol': 3}

In [450]:
DotDict.__setitem__

<function __main__.DotDict.__setitem__(self, key: str, value: Optional[Any])>

In [451]:
test = DotDict(blbl="lol", stuff=74, null_value=None)
print(test)
print(test.null_value)

{'blbl': 'lol', 'stuff': 74, 'null_value': None}
None


In [452]:
dict_test = {
    "param1": "great",
    "param2": 2,
    # "_lol": "lol",
    "param3": {
        "subparam": -5.2,
        # "2": "lol",
        # 1: "haha"
    },
    "param4": [6,1,2]
}
dotdict_test = DotDict(dict_test)
# print()
dotdict_test

{'param1': 'great',
 'param2': 2,
 'param3': {'subparam': -5.2},
 'param4': [6, 1, 2]}

In [453]:
# dir(dotdict_test)

In [454]:
# dotdict_test.__dir__()

In [455]:
# get the dict back
dict_test_returned = dict(dotdict_test)
print(type(dict_test_returned["param3"]))
dict_test_returned

<class 'dict'>


{'param1': 'great',
 'param2': 2,
 'param3': {'subparam': -5.2},
 'param4': [6, 1, 2]}

In [456]:
# accessing values
print(dotdict_test.param3)
print(type(dotdict_test.param3))
print(type(dotdict_test["param3"]))
print(dotdict_test["param4"])
print(hasattr(dotdict_test, "param1"))
print(hasattr(dotdict_test, "fdfd"))
print("param2" in dotdict_test.keys())
print("subparam" in dir(dotdict_test.param3))
print(dotdict_test.param3.subparam)

{'subparam': -5.2}
<class '__main__.DotDict'>
<class '__main__.DotDict'>
[6, 1, 2]
True
False
True
True
-5.2


In [457]:
# setting values
dotdict_test.param5 = {1,2,3}
dotdict_test["param6"] = "blblblb"
# dotdict_test["param7"] = DotDict({
dotdict_test["param7"] = {
    "lol": {
        "Lol0": 0,
        # "1": 2,
        # "_lol3": 6,
        # "_ABC": 4
        # 2: "lol"
    },
    "haha": "prout"
}
# dotdict_test["246"] = 4
dotdict_test

{'param1': 'great',
 'param2': 2,
 'param3': {'subparam': -5.2},
 'param4': [6, 1, 2],
 'param5': {1, 2, 3},
 'param6': 'blblblb',
 'param7': {'lol': {'Lol0': 0}, 'haha': 'prout'}}

In [458]:
dotdict_test.param7.lol.Lol0

0

In [459]:
# delete values
delattr(dotdict_test, "param5")
del dotdict_test["param6"]
dotdict_test

{'param1': 'great',
 'param2': 2,
 'param3': {'subparam': -5.2},
 'param4': [6, 1, 2],
 'param7': {'lol': {'Lol0': 0}, 'haha': 'prout'}}

In [460]:
dotdict_1 = DotDict({
    "a": 0
})
dotdict_2 = DotDict({
    "b": 1
})
dotdict_3 = DotDict({
    "c": 2
})
dict_dotdict = {
    "first": dotdict_1,
    "second": dotdict_2,
    "third": dotdict_3
}
result_dict = dict(DotDict(dict_dotdict))
print(type(result_dict["first"]))
result_dict

<class 'dict'>


{'first': {'a': 0}, 'second': {'b': 1}, 'third': {'c': 2}}

In [461]:
dotdict_test.lol

AttributeError: No key/attribute 'lol'

In [462]:
dotdict_test["haha"]

AttributeError: No key/attribute 'haha'

In [463]:
dotdict_test[0] = "lol"

KeyTypeError: The key '0' is not a string

In [464]:
dotdict_test["0"] = "lol"

AttributeNameError: The key '0' cannot be used as an attribute name

In [465]:
del dotdict_test.lol

AttributeError: No key/attribute 'lol'

In [466]:
del dotdict_test["haha"]

AttributeError: No key/attribute 'haha'

### Class inheritance

`super(type[, instance_or_type])`

- `super` returns a proxy item used to call methods from parent or sibling classes
- `super(type)` returns unbound super object
- `super(type, instance)` instance must be instance of type (isinstance(instance, type))
- `super(type, type2)` type2 must be subclass of type (issubclass(type2, type))

`super` is only used when you don't want to name the parent classes by name => behavior determined at rutime using the MRO

In [159]:
%reset -f
class B:
    def __init__(self, b_atr: int):
        print("init B")
        self.b_atr = b_atr
        
    def meth_b(self):
        print("this is a method of the class B")
        
    # def meth(self):
    #     print("this is the method 'meth' of the class A")

# a_test = A(2)
# print(a_test.a_atr)


class C:
    def __init__(self, c_atr: int):
        print("init C")
        self.c_atr = c_atr
        
    def meth_c(self):
        print("this is a method of the class C")
        
    # def meth(self):
    #     print("this is the method 'meth' of the class B")
        

class D(C):
    def __init__(self, c_atr: int, d_atr: int):
        print("init D")
        # print(super())
        # super().__init__(c_atr)
        C.__init__(self, c_atr)
        self.d_atr = d_atr
        print("end init D")
        
    def meth_d(self):
        print("this is a method of the class D")
        
# d_test = D(1,2)
# print(d_test.d_atr)
# print(d_test.c_atr)
# d_test.meth_d()
# d_test.meth_c()


class A(B, D):
    def __init__(self, a_atr: int, b_atr: int, c_atr: int, d_atr: int):
        print("init A")
        B.__init__(self, b_atr)
        D.__init__(self, c_atr, d_atr)
        self.a_atr = a_atr
        print("end init A")
        
    # def super_tests(self):
    #     print(super())
    #     print(super(self.__class__, self))
    #     # print(super(...))

        
# print("A.__class__:", A.__class__)
# print("type(A):", type(A))
# print("MRO:")
for item in A.__mro__:
    print(item)
print()

a_test = A(1,2,3,4)
print(a_test.a_atr)
print(a_test.b_atr)
print(a_test.c_atr)
print(a_test.d_atr)
a_test.meth_b()
a_test.meth_c()
a_test.meth_d()

<class '__main__.A'>
<class '__main__.B'>
<class '__main__.D'>
<class '__main__.C'>
<class 'object'>

init A
init B
init D
init C
end init D
end init A
1
2
3
4
this is a method of the class B
this is a method of the class C
this is a method of the class D


In [12]:
super(A, B)

<super: __main__.A, __main__.B>

In [5]:
super?

Init signature: super(self, /, *args, **kwargs)
Docstring:     
super() -> same as super(__class__, <first argument>)
super(type) -> unbound super object
super(type, obj) -> bound super object; requires isinstance(obj, type)
super(type, type2) -> bound super object; requires issubclass(type2, type)
Typical use to call a cooperative superclass method:
class C(B):
    def meth(self, arg):
        super().meth(arg)
This works for class methods too:
class C(B):
    @classmethod
    def cmeth(cls, arg):
        super().cmeth(arg)
Type:           type
Subclasses:     

In [40]:
%reset -f

class Square:
    
    def __init__(self, side_length):
        self.side_length = side_length
    
    @property
    def area(self):
        return self.side_length ** 2
        
square_test = Square(3)
print(square_test.side_length)
print(square_test.area)


class Cube(Square):
    
    @property
    def area(self):
        return super().area * 6
    
    @property
    def volume(self):
        return self.side_length ** 3

cube_test = Cube(4)
print(cube_test.side_length)
print(cube_test.area)
print(cube_test.volume)

3
9
4
96
64


In [90]:
%reset -f

class Tokenizer:
    """Tokenize text"""
    def __init__(self, text):
        print('Start Tokenizer init')
        self.tokens = text.split()
        print('End Tokenizer init')


class WordCounter(Tokenizer):
    """Count words in text"""
    def __init__(self, text):
        print('Start WordCounter init')
        super().__init__(text)
        self.word_count = len(self.tokens)
        print('End WordCounter init')


class Vocabulary(Tokenizer):
    """Find unique words in text"""
    def __init__(self, text):
        print('Start Vocabulary init')
        super().__init__(text)
        self.vocab = set(self.tokens)
        print('End Vocabulary init')


class TextDescriber(WordCounter, Vocabulary):
    """Describe text with multiple metrics"""
    def __init__(self, text):
        print('Start TextDescriber init')
        super().__init__(text)
        print('End TextDescriber init')
        
        
# for item in TextDescriber.__mro__:
#     print(item)


td = TextDescriber('row row row your boat')
print('--------')
print(td.tokens)
print(td.vocab)
print(td.word_count)

Start TextDescriber init
Start WordCounter init
Start Vocabulary init
Start Tokenizer init
End Tokenizer init
End Vocabulary init
End WordCounter init
End TextDescriber init
--------
['row', 'row', 'row', 'your', 'boat']
{'your', 'boat', 'row'}
5


In [139]:
# parent, child, grandchild:
%reset -f

class Parent:
    def method(self):
        print("parent method")

class Child(Parent):
    def method(self):
        print("child method")
        print(super())
        super().method()

class Grandchild(Child):
    def method(self):
        print("grandchild method")
        print(super())
        super().method()
    
for item in Grandchild.__mro__:
    print(item)
print()

Grandchild().method()

<class '__main__.Grandchild'>
<class '__main__.Child'>
<class '__main__.Parent'>
<class 'object'>

grandchild method
<super: <class 'Grandchild'>, <Grandchild object>>
child method
<super: <class 'Child'>, <Grandchild object>>
parent method


In [141]:
# child and 2 parents:
%reset -f

class Parent1:
    def method(self):
        print("parent 1 method")
        
class Parent2:
    def method(self):
        print("parent 2 method")
        
class Child(Parent1, Parent2):
    def method(self):
        print("child method")
        print(super())
        print(super(Parent1, self))
        # super(Parent1, self).method()
        # super(Parent2, self).method()
        
for item in Child.__mro__:
    print(item)
print()

Child().method()

<class '__main__.Child'>
<class '__main__.Parent1'>
<class '__main__.Parent2'>
<class 'object'>

child method
<super: <class 'Child'>, <Child object>>
<super: <class 'Parent1'>, <Child object>>
parent 2 method


AttributeError: 'super' object has no attribute 'method'

In [135]:
# Child, Parent1, Parent2, Grandparent:
%reset -f

class Grandparent:
    def method(self):
        print("grandparent method")

class Parent1(Grandparent):
    def method(self):
        print("parent1 method")
        print(super())
        super().method()

class Parent2(Grandparent):
    def method(self):
        print("parent2 method")
        print(super())
        super().method()

class Child(Parent1, Parent2):
    def method(self):
        print("child method")
        print(super())
        # print(super().super())
        super().method()
        
for item in Child.__mro__:
    print(item)
print()

Child().method()

<class '__main__.Child'>
<class '__main__.Parent1'>
<class '__main__.Parent2'>
<class '__main__.Grandparent'>
<class 'object'>

child method
<super: <class 'Child'>, <Child object>>
parent1 method
<super: <class 'Parent1'>, <Child object>>
parent2 method
<super: <class 'Parent2'>, <Child object>>
grandparent method


In [94]:
%reset -f

class Parent1:
    def __init__(self):
        print("parent 1 init")
        
class Parent2:
    def __init__(self):
        print("parent 2 init")
        
class Child(Parent1, Parent2):
    def __init__(self):
        print("child init")
        super().__init__()

Child()

child init
parent 1 init


# DUMP

In [53]:
def test(*args, **kwargs):
    print(args)
    print(kwargs)

args_test = (1,2,3)
kwargs_test = {"lol": 0, "lblb": 1}
test(*args_test, **kwargs_test)

(1, 2, 3)
{'lol': 0, 'lblb': 1}


In [2]:
pattern = re.compile("^[a-zA-Z]\w*$")
pattern

re.compile(r'^[a-zA-Z]\w*$', re.UNICODE)

In [3]:
test_list = [
    "MyClass",
    "_a_hidden_parametter",
    "__dunder__",
    "param_6",
    "YikeS_23",
    "aLBERT2dse",
    "0",
    "25laurence",
    "Myclass#6",
    "This doesn't work either",
    "test's"
]
for str_ in test_list:
    if pattern.match(str_):
        print("match")
    else:
        print("no match")

match
no match
no match
match
match
match
no match
no match
no match
no match
no match


In [90]:
class Test:
    pass
test_dir_return = Test().__dir__()
test_dir_return

['__module__',
 '__dict__',
 '__weakref__',
 '__doc__',
 '__new__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__init__',
 '__reduce_ex__',
 '__reduce__',
 '__getstate__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [91]:
type(test_dir_return)

list

In [205]:
class NullTest:
    def __init__(self):
        self.lol = "kaka"
        print(self.lol)
        # self = None


null_test = NullTest()
null_test.__dict__

kaka


{'lol': 'kaka'}

In [332]:
DotDict({'one': 1, 'three': 3}, two=2)

{'one': 1, 'three': 3, 'two': 2}

In [363]:
test = ["a", "l", "z"]
test.append("e")
test

['a', 'l', 'z', 'e']